In [2]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(plyr)
library(data.table)
library(ggplot2)
library(pheatmap)
library(RColorBrewer)
library(viridis)
source('/home/workspace/IHA_Figure_Revision//helper_function/helper_function_IHA.r')

In [3]:
meta_data=read.csv("/home/workspace/IHA_Figure_Revision/Figure2/04_Composite_Score/Onek1k_dataset_followup/sample_meta_data.csv")
meta_data<-meta_data %>% filter(age>=40, age<=90)

In [4]:
file_list<-paste0("/home/workspace/IHA_Figure_Revision/Figure2/04_Composite_Score/Onek1k_dataset_followup/sample_normalized_count_average/",meta_data$donor_id,".csv")

df_list<-read_pseudobulk_expression(file_list)


[1] "Total reading time: 43.201 seconds"
[1] "The length of the list matches the length of the input path."


In [5]:
df<-read.csv("/home/workspace/IHA_Figure_Revision/Figure1/02_DEG/Deseq2_Result_Y1D0.csv")

In [6]:
df<-df %>% filter(contrast=='cohort.cohortGuid')

In [7]:
df_subset<-df %>% filter(abs(log2FoldChange)>0.1,padj<0.05)

In [8]:
selected_celltype<-read.csv("/home/workspace/IHA_Figure_Revision/Figure1/04_CompositeScore/BRI_CompositeScore_Down.csv")

In [9]:
keep_celltypes <- unique(selected_celltype$celltype)


In [10]:
df_subset_filtered<-df_subset%>% filter(celltype %in% keep_celltypes)


In [11]:
meta_data$Ages<-meta_data$age

In [12]:
meta_data<-meta_data %>% arrange(Ages)
meta_data$pbmc_sample_id_gsubbed<-gsub("-",'.',meta_data$donor_id) 
rownames(meta_data)<-meta_data$pbmc_sample_id_gsubbed
meta_data$pbmc_sample_id_gsubbed<-paste0("X",meta_data$pbmc_sample_id_gsubbed)


In [13]:
max(meta_data$Ages)

[1] 90

In [12]:
Up_Genes<-c('SESN3','PTGER2')

In [13]:
exp_all<-list()
for (gene_single in Up_Genes) {
  celltypes <- unique(df_subset_filtered %>% filter(gene == gene_single) %>%
                      arrange(celltype)%>% select(celltype) %>% pull())
  exp_single_gene_all_celtype <- data.frame()
  for (celltype_single in celltypes) {
    exp <- lapply(df_list, function(df) {
      df <- df[gene_single, grep(celltype_single, names(df), fixed = TRUE), drop = FALSE]
    })

    exp_matrix <- do.call(cbind, exp)
    colnames(exp_matrix) <- sub(":.*", "", colnames(exp_matrix))
    
    # Convert to data frame and retain row names as a new column
    exp_df <- data.frame(exp_matrix)
    exp_df<-as.data.frame(t(exp_df))
    colnames(exp_df)<-"mean_exp"
    
    exp_df$pbmc_sample_id_gsubbed<-rownames(exp_df)
    exp_df<-left_join(exp_df,meta_data)
    
    
    
    exp_df<-exp_df %>% group_by(Ages) %>% dplyr::summarise(mean_exp=mean(mean_exp) )
    
    exp_df<-as.data.frame(exp_df)
    
    rownames(exp_df)<-exp_df$Ages
    
    exp_df<-as.data.frame(t(exp_df[2])  )
    rownames(exp_matrix)<-gene_single
    exp_df$rownames <- paste0(rownames(exp_matrix), ":", celltype_single)

    # Reorder to have rownames first (optional)
    exp_df <- exp_df[, c("rownames", setdiff(names(exp_df), "rownames"))]

    # Bind the data while keeping row names as a column
    exp_single_gene_all_celtype <- rbind.fill(exp_single_gene_all_celtype, exp_df)
  }
  
  # Optionally set the row names back if needed
rownames(exp_single_gene_all_celtype) <- exp_single_gene_all_celtype$rownames
exp_single_gene_all_celtype <- exp_single_gene_all_celtype[, -1]  # Remove the 'rownames' column after setting row names

exp_all[[gene_single]]<-exp_single_gene_all_celtype
}


Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`
Joining with `by = join_by(pbmc_sample_id_gsubbed)`


In [14]:
df<-do.call(rbind,exp_all)

In [16]:
write.csv(df,"Up_Selected_Onek1K.csv")